In [13]:
#r "nuget: LanguageExt.Core, 4.4.0"


#!value --name strategyGuide --from-file strategy.txt

Installed Packages LanguageExt.Core, 4.4.0

In [22]:
using System;
using LanguageExt;

readonly record struct OpponentChoice(string Value)
{
   public static implicit operator string(OpponentChoice value) => value.Value;
   public static implicit operator OpponentChoice(string value) => new(value);
}

readonly record struct PredictedChoice(string Value)
{
   public static implicit operator string(PredictedChoice value) => value.Value;
   public static implicit operator PredictedChoice(string value) => new(value);
}

readonly record struct Round(OpponentChoice opponent, PredictedChoice prediction);

var scorePrediction = (PredictedChoice p) => p.Value switch
{
    "X" => 1,
    "Y" => 2,
    "Z" => 3,
    _ => -1
};

var scoreRound = (OpponentChoice o) => (PredictedChoice p) => new { Them = o.Value, Me = p.Value } switch
{
    { Them: "A", Me: "X" } => 3,
    { Them: "B", Me: "Y" } => 3,
    { Them: "C", Me: "Z" } => 3,
    { Them: "A", Me: "Z" } => 0,
    { Them: "B", Me: "X" } => 0,
    { Them: "C", Me: "Y" } => 0,
    { Them: "C", Me: "X" } => 6,
    { Them: "A", Me: "Y" } => 6,
    { Them: "B", Me: "Z" } => 6,
    _ => -1
};

var totalScoreForRound = (OpponentChoice o) => (PredictedChoice p) =>
    scorePrediction(p) + scoreRound(o)(p);

var scoreGame = (string strategyGuide) =>
    strategy
    .Split(Environment.NewLine)
    .Select(s => {
        OpponentChoice o = s.Split(' ').First();
        PredictedChoice p = s.Split(' ').Skip(1).First();
        return new Round(o, p);
    })
    .Select(r => totalScoreForRound(r.opponent)(r.prediction))
    .Sum();

In [23]:
// proof on sample data separator, int cou

var sampleStrategy =
"""
A Y
B X
C Z
""";

var totalScore = sampleStrategy
    .Split(Environment.NewLine)
    .Select(s => {
        OpponentChoice o = s.Split(' ').First();
        PredictedChoice p = s.Split(' ').Skip(1).First();
        return new Round(o, p);
    })
    .Select(r => totalScoreForRound(r.opponent)(r.prediction))
    .Sum();

totalScore == 15

True